In [2]:
from bs4 import BeautifulSoup
import datetime, json
import numpy as np
import pandas as pd
import requests, time
import warnings
warnings.simplefilter('ignore')

import talib as ta
from talib import MA_Type

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn import metrics, svm

import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter
import matplotlib.ticker as ticker